In [12]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [9]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Points for Businesses and Attractions
`civicknowledge.com-osmpoints-demosearch-1.1.1` Last Update: 2021-05-11T19:17:40

_Points for businesses and attractions, from OSM, aggregated to approx 300m._




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong>points</string>, <em>index:civicknowledge.com-osm-demosearch#point_tags</em>. OSM Points</li><li> <strong>utm_bands_buffered</string>, <em>index:civicknowledge.com-mgrs#utm_bands_buffered</em>. Buffered UTM bands</li><ul>

In [3]:
op = pkg.reference('points').geoframe()
op.head()

,geohash,geoid,amenity,tourism,shop,leisure,natural,parking,bank,bar,...,fuel,grave_yard,hotel,laundry,park,parking_space,playground,restaurant,supermarket,geometry
0,87vg4y02,31000US28180,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,POINT (-160.54270 21.65414)
1,87vg4ycq,31000US28180,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,POINT (-160.54132 21.65929)
2,87vg4z47,31000US28180,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,POINT (-160.53995 21.66015)
3,87y5cz1h,31000US28180,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,POINT (-160.23405 21.79215)
4,87y5fkz8,31000US28180,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,POINT (-160.20315 21.77911)


In [21]:
op.head().T

,0,1,2,3,4
geohash,87vg4y02,87vg4ycq,87vg4z47,87y5cz1h,87y5fkz8
geoid,31000US28180,31000US28180,31000US28180,31000US28180,31000US28180
amenity,0,0,0,0,0
tourism,0,0,0,0,0
shop,0,0,0,0,0
leisure,0,0,0,0,0
natural,1,1,1,1,1
parking,0,0,0,0,0
bank,0,0,0,0,0
bar,0,0,0,0,0


In [10]:

utm = pkg.reference('utm_bands_buffered').geoframe()
utm.head()

,band,geometry
0,10,"POLYGON ((-119.88462 38.99997, -119.88620 37.9..."
1,11,"POLYGON ((-113.88462 38.99997, -113.88620 37.9..."
2,12,"POLYGON ((-107.89643 29.99999, -107.89744 29.0..."
3,13,"POLYGON ((-101.89643 29.99999, -101.89744 29.0..."
4,14,"POLYGON ((-95.89643 29.99999, -95.89744 29.000..."


In [23]:
sp = gpd.sjoin(op, utm)

# Aggregate groups
groups = {
    'amenity': ['amenity'],
    'cafe': ['cafe'],
    'restaurant': ['restaurant'],
    'bar': ['bar'],
    'entertain': ['cafe', 'restaurant', 'bar'],
    'casual': ['fast_food', 'convenience'],
    'food': ['convenience','supermarket'],
    'shop': ['shop', 'clothes', 'supermarket', 'bank', 'laundry', 'parking'],
    'active': ['playground', 'bicycle_parking', 'fitness_centre', 'park'],
    'playpark': ['playground', 'park'],
    'travel': ['fuel', 'hotel', 'amenity', 'tourism', 'leisure', 'natural'],
}

for agg, cols in groups.items():
    # Reduce all of the layers to 1 per geohash. Any more than that is probably spurious
    sp.loc[:, cols] = (sp.loc[:, cols] > 0).astype(np.int8)
    sp[agg] = sp[cols].sum(axis=1)

sp.loc[:, groups.keys()] = sp.loc[:, groups.keys()].mask(sp.loc[:, groups.keys()] > 1, 1)

In [24]:
points = sp.loc[:, ['band', 'geometry'] + list(groups.keys())]


In [25]:
points

,band,geometry,amenity,cafe,restaurant,bar,entertain,casual,food,shop,active,playpark,travel
95889,12,POINT (-113.04125 24.33446),0,0,0,0,0,0,0,0,0,0,1
95890,12,POINT (-113.57374 26.71214),1,0,1,0,1,0,0,0,0,0,1
95891,12,POINT (-112.51287 26.37019),0,0,0,0,0,0,0,1,0,0,0
95902,12,POINT (-114.01251 27.74863),0,0,0,0,0,0,0,0,0,0,1
95903,12,POINT (-114.01182 27.74932),0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013707,19,POINT (-66.00466 47.91713),1,0,0,0,0,0,0,0,0,0,1
2013708,19,POINT (-65.97754 47.92125),0,0,0,0,0,0,0,0,0,0,1
2013709,19,POINT (-65.94870 47.91318),0,0,0,0,0,0,0,1,0,0,0
2013710,19,POINT (-65.94320 47.92039),0,0,0,0,0,0,0,0,0,0,1
